In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import torch.optim as optim

In [2]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root = '../data',train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size= 32, shuffle=False, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform= transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=0)

classes = ('plane','car','bird','cat','deer','dog','frong','horse','ship','truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class BasicBlock(nn.Module):
    def __init__(self, inchannel, channel, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(inchannel, channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channel)
        self.conv2 = nn.Conv2d(channel, channel, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channel)
        self.downsample = downsample
        
    def forward(self, x):
        shortcut = x
        
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        if self.downsample is not None:
            shortcut = self.downsample(x)
            
        out += shortcut
        out = F.relu(out)
        
        return out

In [4]:
class ResNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 16
        self.conv1 = nn.Conv2d(3,16,kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self.make_layer(BasicBlock, 16, 3)
        self.layer2 = self.make_layer(BasicBlock, 32, 3, stride=2)
        self.layer3 = self.make_layer(BasicBlock, 64, 3, stride=2)
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, channel, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inchannel != channel:
            downsample = nn.Sequential(nn.Conv2d(self.inchannel, channel, kernel_size=1, stride=stride, bias=False),
                                      nn.BatchNorm2d(channel),)
            
        layers = []
        layers.append(block(self.inchannel, channel, stride, downsample))
        self.inchannel = channel
        for i in range(1, blocks):
            layers.append(block(self.inchannel, channel))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        
        return x

In [5]:
net = ResNet()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.device_count() > 1:
    print("Let's use ",torch.cuda.device_count(),"GPUs!")
    net = nn.DataParallel(net)
    
net.to(device)

Let's use  2 GPUs!


DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.1, weight_decay= 0.0001, momentum=0.9)

In [7]:
def train(epoch):
    net.train()
    for batch_idx,(imgs, labels) in enumerate(trainloader):
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = net(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(imgs), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.data[0]))
            
    print("Training finished")

In [8]:
def test():
    net.eval()
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            c = (predicted == labels).squeeze()
        
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    total_correct =  0
    for i in range(10):
        total_correct += 100*class_correct[i]/class_total[i]
        print('Accuracy of %5s : %2d %%'%(classes[i], 100*class_correct[i]/class_total[i]))

    print('Accuracy overall: %d' % (total_correct/10))
    
    """
    with torch.no_grad():
        for imgs, labels in testloader:
            imgs,labels = imgs.to(device), labels.to(device)

            outputs = net(imgs)
            test_loss += criterion(outputs,labels).data[0]
            pred = outputs.data.max(1, keepdim=True)[1]
            correct += pred.eq(labels.data.view_as(pred)).cpu().sum()

        test_loss /= len(testloader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))
    """

In [9]:
for epoch in range(1,100):
    train(epoch)
    test()

/home/hsue/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.281666
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.671543
Training finished
Accuracy of plane : 11 %
Accuracy of   car : 49 %
Accuracy of  bird : 36 %
Accuracy of   cat : 44 %
Accuracy of  deer : 43 %
Accuracy of   dog :  5 %
Accuracy of frong : 57 %
Accuracy of horse : 73 %
Accuracy of  ship : 93 %
Accuracy of truck : 76 %
Accuracy overall: 49
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.277854
Train Epoch: 2 [32000/50000 (64%)]	Loss: 1.240205
Training finished
Accuracy of plane : 47 %
Accuracy of   car : 74 %
Accuracy of  bird : 59 %
Accuracy of   cat : 48 %
Accuracy of  deer : 60 %
Accuracy of   dog : 31 %
Accuracy of frong : 61 %
Accuracy of horse : 84 %
Accuracy of  ship : 91 %
Accuracy of truck : 95 %
Accuracy overall: 65
Train Epoch: 3 [0/50000 (0%)]	Loss: 0.703081
Train Epoch: 3 [32000/50000 (64%)]	Loss: 0.841359
Training finished
Accuracy of plane : 50 %
Accuracy of   car : 72 %
Accuracy of  bird : 63 %
Accuracy of   cat : 55 %
Accuracy of  deer :

Accuracy of plane : 80 %
Accuracy of   car : 82 %
Accuracy of  bird : 73 %
Accuracy of   cat : 85 %
Accuracy of  deer : 78 %
Accuracy of   dog : 55 %
Accuracy of frong : 60 %
Accuracy of horse : 83 %
Accuracy of  ship : 89 %
Accuracy of truck : 82 %
Accuracy overall: 77
Train Epoch: 23 [0/50000 (0%)]	Loss: 0.119915
Train Epoch: 23 [32000/50000 (64%)]	Loss: 0.188983
Training finished
Accuracy of plane : 76 %
Accuracy of   car : 93 %
Accuracy of  bird : 78 %
Accuracy of   cat : 77 %
Accuracy of  deer : 80 %
Accuracy of   dog : 46 %
Accuracy of frong : 66 %
Accuracy of horse : 84 %
Accuracy of  ship : 92 %
Accuracy of truck : 83 %
Accuracy overall: 77
Train Epoch: 24 [0/50000 (0%)]	Loss: 0.120223
Train Epoch: 24 [32000/50000 (64%)]	Loss: 0.302858
Training finished
Accuracy of plane : 70 %
Accuracy of   car : 95 %
Accuracy of  bird : 82 %
Accuracy of   cat : 79 %
Accuracy of  deer : 77 %
Accuracy of   dog : 53 %
Accuracy of frong : 66 %
Accuracy of horse : 86 %
Accuracy of  ship : 92 %
Acc

Train Epoch: 44 [32000/50000 (64%)]	Loss: 0.190873
Training finished
Accuracy of plane : 79 %
Accuracy of   car : 93 %
Accuracy of  bird : 72 %
Accuracy of   cat : 67 %
Accuracy of  deer : 82 %
Accuracy of   dog : 56 %
Accuracy of frong : 73 %
Accuracy of horse : 92 %
Accuracy of  ship : 95 %
Accuracy of truck : 75 %
Accuracy overall: 78
Train Epoch: 45 [0/50000 (0%)]	Loss: 0.163762
Train Epoch: 45 [32000/50000 (64%)]	Loss: 0.136501
Training finished
Accuracy of plane : 85 %
Accuracy of   car : 93 %
Accuracy of  bird : 74 %
Accuracy of   cat : 74 %
Accuracy of  deer : 73 %
Accuracy of   dog : 61 %
Accuracy of frong : 63 %
Accuracy of horse : 90 %
Accuracy of  ship : 92 %
Accuracy of truck : 81 %
Accuracy overall: 79
Train Epoch: 46 [0/50000 (0%)]	Loss: 0.195574
Train Epoch: 46 [32000/50000 (64%)]	Loss: 0.421597
Training finished
Accuracy of plane : 86 %
Accuracy of   car : 87 %
Accuracy of  bird : 69 %
Accuracy of   cat : 75 %
Accuracy of  deer : 79 %
Accuracy of   dog : 60 %
Accuracy 

Train Epoch: 66 [32000/50000 (64%)]	Loss: 0.172133
Training finished
Accuracy of plane : 81 %
Accuracy of   car : 93 %
Accuracy of  bird : 75 %
Accuracy of   cat : 74 %
Accuracy of  deer : 65 %
Accuracy of   dog : 61 %
Accuracy of frong : 71 %
Accuracy of horse : 93 %
Accuracy of  ship : 94 %
Accuracy of truck : 86 %
Accuracy overall: 79
Train Epoch: 67 [0/50000 (0%)]	Loss: 0.181936
Train Epoch: 67 [32000/50000 (64%)]	Loss: 0.248527
Training finished
Accuracy of plane : 84 %
Accuracy of   car : 93 %
Accuracy of  bird : 82 %
Accuracy of   cat : 79 %
Accuracy of  deer : 81 %
Accuracy of   dog : 51 %
Accuracy of frong : 64 %
Accuracy of horse : 76 %
Accuracy of  ship : 92 %
Accuracy of truck : 82 %
Accuracy overall: 78
Train Epoch: 68 [0/50000 (0%)]	Loss: 0.206658
Train Epoch: 68 [32000/50000 (64%)]	Loss: 0.248811
Training finished
Accuracy of plane : 87 %
Accuracy of   car : 86 %
Accuracy of  bird : 78 %
Accuracy of   cat : 71 %
Accuracy of  deer : 72 %
Accuracy of   dog : 61 %
Accuracy 

Train Epoch: 88 [32000/50000 (64%)]	Loss: 0.355958
Training finished
Accuracy of plane : 77 %
Accuracy of   car : 94 %
Accuracy of  bird : 82 %
Accuracy of   cat : 79 %
Accuracy of  deer : 71 %
Accuracy of   dog : 55 %
Accuracy of frong : 68 %
Accuracy of horse : 77 %
Accuracy of  ship : 94 %
Accuracy of truck : 68 %
Accuracy overall: 77
Train Epoch: 89 [0/50000 (0%)]	Loss: 0.217674
Train Epoch: 89 [32000/50000 (64%)]	Loss: 0.174415
Training finished
Accuracy of plane : 81 %
Accuracy of   car : 97 %
Accuracy of  bird : 75 %
Accuracy of   cat : 67 %
Accuracy of  deer : 78 %
Accuracy of   dog : 53 %
Accuracy of frong : 82 %
Accuracy of horse : 89 %
Accuracy of  ship : 85 %
Accuracy of truck : 77 %
Accuracy overall: 78
Train Epoch: 90 [0/50000 (0%)]	Loss: 0.081700
Train Epoch: 90 [32000/50000 (64%)]	Loss: 0.417766
Training finished
Accuracy of plane : 78 %
Accuracy of   car : 93 %
Accuracy of  bird : 86 %
Accuracy of   cat : 65 %
Accuracy of  deer : 67 %
Accuracy of   dog : 61 %
Accuracy 